# Path Planning
## Outdoor Routing

### Instructions:

- first get the your `current location` (latitude and longitude) 
- then, set them by using `set_curr_location(long,lat)` function
- then, you can set your distnation location by using `set_dist_location(long,lat)` function
- after that use the function: `get_directions()` without any parameters and it will get the direstion and the distance from your location untill reach the distnation.
- finally call the function: `converting_list(list)` and pass the output list from the previous one and its output will be the final instraction which will be sent to the embedded system directly. 

### bouns functions:
- `get_curr_location()` which return your current location
- `get_dist_location()` which return your distination location

## 1- GEO locations for current and distination

In [1]:
# import libraries
from geopy.geocoders import Nominatim
import time
from pprint import pprint
import requests
from ratelimiter import RateLimiter
import geopy

In [2]:
# set the current location from GPS Module
#  29.414932, 30.862395

global Curr_long 
global Curr_lat 

Curr_long = 29.414932
Curr_lat = 30.862395


In [3]:
def set_curr_location(long,lat):
    global Curr_long 
    Curr_long = long
    global Curr_lat
    Curr_lat = lat

def get_curr_location():
    global Curr_long  
    global Curr_lat
    return Curr_long, Curr_lat    

### Test

In [4]:

set_curr_location(29.408314, 30.865541)
#print(Curr_long )

In [5]:
long, lat = get_curr_location()
print(long)
print(lat)
# or by 
print(Curr_long)
print(Curr_lat)

29.408314
30.865541
29.408314
30.865541


## 2- Set distination


In [6]:
# set the distination location from GPS Module

global dist_long 
dist_long = 30.414932
global dist_lat 
dist_lat = 30.862395

def set_dist_location(long,lat):
    global dist_long 
    dist_long = long
    global dist_lat
    dist_lat = lat

def get_dist_location():
    global dist_long  
    global dist_lat
    return dist_long, dist_lat    

### Test

In [7]:
set_dist_location(29.412674, 30.976626)

In [8]:
long, lat = get_dist_location()
print(long)
print(lat)

29.412674
30.976626


## 3- Using directions api

In [9]:
def get_directions():
    clong, clat = get_curr_location()
    dlong, dlat = get_dist_location()
    
    url = "https://trueway-directions2.p.rapidapi.com/FindDrivingPath"
    org ="29.408314, 30.865541"
    dis ="29.412674, 30.876626"
    querystring = {"origin": str(clong)+","+ str(clat) ,"destination":str(dlong)+","+ str(dlat)}

    headers = {
        "X-RapidAPI-Host": "trueway-directions2.p.rapidapi.com",
        "X-RapidAPI-Key": "a5c9d69029mshecf871aedf5feddp11fc8fjsn15ca8e0cec3c"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    direc = response.text
    dir_json = response.json()
    list_json = dir_json['route']['steps']
    
    dirc_list =[]
    dist_list =[]
    list =[]
  
    for i in range(len(list_json)):
        #print(i)
        try:
            if i==0:
                dirc_list.append('forward')
                dist_list.append(list_json[i]['distance'])
                #list['forward']=list_json[i]['distance']
                #print('forward')
            else:
                dirc_list.append(list_json[i]['maneuver'])
                dist_list.append(list_json[i]['distance'])
                #list[list_json[i]['maneuver']]=list_json[i]['distance']
        except Exception as e:
            dirc_list.append('forward')
            dist_list.append(list_json[i]['distance'])
            #list['forward']=list_json[i]['distance']
    
    list.append(dirc_list)
    list.append(dist_list)

    return list 

In [10]:
def converting_list(list):
    """replace the directions and all the instrations like \"turn right\"
    and \"roundabout right \" to just 4 charactries to be sent to the embedded system 
    """
    new_dirc_list = []
    new_dist_list = []
    new_list = []
    
    for i in range(len(list[0])):
        if list[0][i] =='forward':
            new_dirc_list.append('f')
            new_dist_list.append(list[1][i])
            
        elif list[0][i] =='turn right':
            new_dirc_list.append('r')
            new_dist_list.append(0)
            new_dirc_list.append('f')
            new_dist_list.append(list[1][i])
            
        elif list[0][i] =='turn left':
            new_dirc_list.append('l')
            new_dist_list.append(0)
            new_dirc_list.append('f')
            new_dist_list.append(list[1][i])
            
        elif list[0][i] =='uturn right':
            new_dirc_list.append('r')
            new_dist_list.append(0)
            new_dirc_list.append('f')
            new_dist_list.append(4)
            new_dirc_list.append('r')
            new_dist_list.append(0)
            new_dirc_list.append('f')
            new_dist_list.append(list[1][i])
            
        elif list[0][i] =='uturn left':
            new_dirc_list.append('l')
            new_dist_list.append(0)
            new_dirc_list.append('f')
            new_dist_list.append(4)
            new_dirc_list.append('l')
            new_dist_list.append(0)
            new_dirc_list.append('f')
            new_dist_list.append(list[1][i])
            
        elif list[0][i] =='roundabout right':
            new_dirc_list.append('r')
            new_dist_list.append(0)
            new_dirc_list.append('f')
            new_dist_list.append(5)
            new_dirc_list.append('r')
            new_dist_list.append(0)
            new_dirc_list.append('f')
            new_dist_list.append(list[1][i])
            
        elif list[0][i] =='roundabout left':
            new_dirc_list.append('l')
            new_dist_list.append(0)
            new_dirc_list.append('f')
            new_dist_list.append(5)
            new_dirc_list.append('l')
            new_dist_list.append(0)
            new_dirc_list.append('f')
            new_dist_list.append(list[1][i])
            
        elif list[0][i] =='turn slight right':
            new_dirc_list.append('r')
            new_dist_list.append(0)
            new_dirc_list.append('f')
            new_dist_list.append(list[1][i])
            
        elif list[0][i] =='turn slight left':
            new_dirc_list.append('l')
            new_dist_list.append(0)
            new_dirc_list.append('f')
            new_dist_list.append(list[1][i])
    
    new_list.append(new_dirc_list)
    new_list.append(new_dist_list)

    return new_list

- uturn left => turn left 90d then go f 4m then turn left agian
- roundabout right => turn right then go f 5m then turn right agian
- turn slight left => turn left

### Test

In [11]:
list_json=get_directions()

list_json


[['forward',
  'uturn left',
  'turn right',
  'roundabout right',
  'turn right',
  'turn right',
  'uturn left',
  'turn right',
  'forward',
  'turn right',
  'turn left',
  'turn left',
  'turn right',
  'turn left',
  'turn left',
  'turn right',
  'turn slight left'],
 [68,
  31,
  308,
  678,
  448,
  165,
  171,
  1131,
  2097,
  15,
  286,
  314,
  172,
  5700,
  1954,
  107,
  22]]

In [12]:
converting_list(list_json)

[['f',
  'l',
  'f',
  'l',
  'f',
  'r',
  'f',
  'r',
  'f',
  'r',
  'f',
  'r',
  'f',
  'r',
  'f',
  'l',
  'f',
  'l',
  'f',
  'r',
  'f',
  'f',
  'r',
  'f',
  'l',
  'f',
  'l',
  'f',
  'r',
  'f',
  'l',
  'f',
  'l',
  'f',
  'r',
  'f',
  'l',
  'f'],
 [68,
  0,
  4,
  0,
  31,
  0,
  308,
  0,
  5,
  0,
  678,
  0,
  448,
  0,
  165,
  0,
  4,
  0,
  171,
  0,
  1131,
  2097,
  0,
  15,
  0,
  286,
  0,
  314,
  0,
  172,
  0,
  5700,
  0,
  1954,
  0,
  107,
  0,
  22]]